In [1]:
# required modules
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('default')

In [2]:
df_rest = pd.read_csv('../data/data_combined_v1.03.csv')
df_rest.head()

,place_id,tripadvisor_id,fsq_id,name,business_status,types,price_level,ta_price_level,price_level_combined,rating,...,population_20_24,population_25_34,population_35_44,population_45_54,population_54_64,population_65+,resident_per_km2,brut_income_annual_per_person,district,district_cluster
0,ChIJd6kh67ZkjEcRC1IQourQOyo,7159041.0,4de685281f6e9d90c7f5325c,Restaurant La Tentation,OPERATIONAL,"['restaurant', 'point_of_interest', 'food', 'e...",2.0,2.0,3.0,4.2,...,1937,5163,5549,4924,4290,5625,16282,109266,Saint-Jean Charmilles,3
1,ChIJX68uubZkjEcRKKxpZ7OrgqU,13322506.0,NaN,Café Restaurant Palma,OPERATIONAL,"['restaurant', 'point_of_interest', 'food', 'e...",NaN,NaN,4.0,3.7,...,1937,5163,5549,4924,4290,5625,16282,109266,Saint-Jean Charmilles,3
2,ChIJPcYsG_BljEcR4kTFqBJJoK4,6720272.0,NaN,Le Versailles,OPERATIONAL,"['restaurant', 'point_of_interest', 'food', 'e...",NaN,2.0,3.0,3.9,...,1937,5163,5549,4924,4290,5625,16282,109266,Saint-Jean Charmilles,3
3,ChIJV8uySFxljEcRn7XpIO3zZX0,NaN,NaN,Restaurant Les Jardins du Rhône,OPERATIONAL,"['restaurant', 'food', 'point_of_interest', 'e...",NaN,NaN,NaN,NaN,...,1937,5163,5549,4924,4290,5625,16282,109266,Saint-Jean Charmilles,3
4,ChIJW8zU8rZkjEcR_Ke8t7c7hVc,NaN,NaN,Canteen De La Residence Gardens Du Rhône,OPERATIONAL,"['restaurant', 'food', 'point_of_interest', 'e...",NaN,NaN,4.0,5.0,...,1937,5163,5549,4924,4290,5625,16282,109266,Saint-Jean Charmilles,3


In [11]:
df_rest.columns

Index(['place_id', 'tripadvisor_id', 'fsq_id', 'name', 'business_status',
       'types', 'price_level', 'ta_price_level', 'price_level_combined',
       'rating', 'user_ratings_total', 'ta_rating', 'ta_user_ratings_total',
       'combined_rating', 'geometry.location.lat', 'geometry.location.lng',
       'formatted_address', 'zip_code', 'city', 'international_phone_number',
       'website', 'google_category', 'ta_4sq_cuisine1', 'ta_4sq_cuisine2',
       'ta_4sq_cuisine3', 'ta_4sq_cuisine4', 'ta_4sq_cuisine5',
       'main_category_1', 'main_category_2', 'main_category_3',
       'main_category_4', 'combined_main_category', 'sub_category',
       'neighboorhood_area_km2', 'resident_population_total',
       'population_age_0_14', 'population_15_19', 'population_20_24',
       'population_25_34', 'population_35_44', 'population_45_54',
       'population_54_64', 'population_65+', 'resident_per_km2',
       'brut_income_annual_per_person', 'district', 'district_cluster'],
      dtype='o

In [98]:
df_lite = df_rest[[
    'place_id',
    'name',
    'price_level_combined',
    'combined_rating',
    'geometry.location.lat', 'geometry.location.lng',
    'combined_main_category',
    'sub_category',
    'district', 
    'district_cluster']]

In [108]:
def analyze_district(df, rest_category=None):
    if rest_category:
        df = df[df['combined_main_category'] == rest_category]
        
    most_comp_dist = df.groupby('district')[['place_id']].count().sort_values(by='place_id',ascending=False).head(3)
    least_comp_dist = df.groupby('district')[['place_id']].count().sort_values(by='place_id',ascending=True).head(3)
    
    display(most_comp_dist)
    display(least_comp_dist)

In [109]:
# Testing w/o restaurant category
analyze_district(df_lite)

,place_id
district,
Cité-Centre,277
Pâquis Sécheron,260
Grottes Saint-Gervais,252


,place_id
district,
Bâtie - Acacias,49
Champel,55
Servette Petit-Saconnex,57


In [110]:
# Testing with restaurant category
analyze_district(df_lite, 'Italian')

,place_id
district,
Cité-Centre,16
Eaux-Vives - Lac,13
Grottes Saint-Gervais,10


,place_id
district,
Bâtie - Acacias,2
Champel,2
Servette Petit-Saconnex,2


In [111]:
# Testing with restaurant category
analyze_district(df_lite, 'Mexican')

,place_id
district,
Grottes Saint-Gervais,6
Jonction - Plainpalais,5
La Cluse - Philosophes,3


,place_id
district,
Cité-Centre,1
Saint-Jean Charmilles,1
Eaux-Vives - Lac,2


In [135]:
def analyze_cluster(df, rest_district=None, rest_category=None):
    if rest_district and rest_category:
        df = df[(df['combined_main_category'] == rest_category) & (df['district'] == rest_district)]
        
    elif rest_district:
        df = df[df['district'] == rest_district]
        
    elif rest_category:
        df = df[df['combined_main_category'] == rest_category]
    else:
        df = df
        
    most_comp_clus = df.groupby('district_cluster')[['place_id']].count().sort_values(by='place_id',ascending=False).head(5)
    least_comp_clus = df.groupby('district_cluster')[['place_id']].count().sort_values(by='place_id',ascending=True).head(5)
    
    display(most_comp_clus)
    display(least_comp_clus)

In [136]:
# without district & restaurant type
analyze_cluster(df, rest_district=None, rest_category=None)

,place_id
district_cluster,
28,98
36,89
39,86
26,77
21,74


,place_id
district_cluster,
8,1
43,4
3,7
41,8
30,8


In [137]:
# without district & with rest_type
analyze_cluster(df, rest_district=None, rest_category='Italian')

,place_id
district_cluster,
39,7
17,7
21,4
27,4
38,3


,place_id
district_cluster,
1,1
31,1
41,1
18,1
15,1


In [138]:
# with district & with rest_type
analyze_cluster(df, rest_district="Cité-Centre", rest_category='Italian')

,place_id
district_cluster,
39,7
36,3
37,3
38,3


,place_id
district_cluster,
36,3
37,3
38,3
39,7


In [139]:
# with district & with rest_type
analyze_cluster(df, rest_district="Cité-Centre", rest_category='Mexican')

,place_id
district_cluster,
38,1


,place_id
district_cluster,
38,1


In [144]:
# with district & with rest_type
analyze_cluster(df, rest_district="Champel", rest_category='Mexican')

,place_id
district_cluster,


,place_id
district_cluster,


Main Page

- There xx is rest in GVA
    - Most common xx rest, followed by yy
    - Least common one xx rest, followed by yy

- Most restaurants are in aa district followed by bb
- Least restaurants are in cc followed by

When you select 
District Page
   - There xx is rest in GVA
   - Most common xx rest, followed by yy
   - Least common one xx rest, followed by yy
    
   - Most densed area xx
   - Least densed area yy
   
Should be similar when you select restaurant